# [공빅데] CCTV,가로등 위치를 folium을 사용하여 clustering 시각화하기

### step1. 데이터 준비하기
[용산구 CCTV 데이터](https://data.seoul.go.kr/dataList/OA-20926/S/1/datasetView.do)  
[전국보안등정보표준데이터](https://www.data.go.kr/data/15017320/standard.do)

### step2. cctv와 가로등 위치데이터 추출하기

In [28]:
### cctv 데이터 ###
import pandas as pd
cctv_csv = pd.read_csv('pydata/yongsan_CCTV.csv', encoding = 'cp949')

# nan 데이터 0으로 변환
cctv_csv = cctv_csv.fillna(0)

# x좌표, y좌표 리스트로 만들기
x = []
y = []

for i in range(len(cctv_csv['WGS x좌표'])):
    if cctv_csv['WGS x좌표'][i] == 0 or cctv_csv['WGS y좌표'][i] == 0: # 위경도 값이 0인 곳은 제외
        pass
    else:
        x.append(cctv_csv['WGS x좌표'][i])
        y.append(cctv_csv['WGS y좌표'][i])
        
print('x개수:', len(x))
print('y개수:', len(y))

x개수: 743
y개수: 743


In [29]:
### 가로등 데이터 ###
street_csv = pd.read_csv('pydata/서울특별시_용산구_보안등정보.csv', encoding = 'cp949')

# nan 데이터 0으로 변환
street_csv = street_csv.fillna(0)

# x좌표, y좌표 리스트로 만들기
x1 = []
y1 = []
name = []

for i in range(len(street_csv['위도'])):
    if street_csv['위도'][i] == 0 or street_csv['경도'][i] == 0: # 위경도 값이 0인 곳은 제외
        pass
    else:
        x1.append(street_csv['위도'][i])
        y1.append(street_csv['경도'][i])
        name.append(street_csv['보안등위치명'][i])
        
print('x개수:', len(x1))
print('y개수:', len(y1))
print('name개수:', len(name))

x개수: 9998
y개수: 9998
name개수: 9998


### step3. 지도를 만들고 그 위에 위치 표시하기

In [35]:
import folium
import folium.plugins as plug
import json

# map 만들기
map_osm = folium.Map(location=[37.538821, 126.98368841130002],zoom_start=14)
#geojson파일 오픈
with open('pydata/yongsan.zip.geojson', 'rt') as f:
    geo = json.load(f)
    f.close()
folium.GeoJson(geo, name='YongSan').add_to(map_osm)
map_osm

In [48]:
###가로등, cctv 위치 표시및 클러스터링### 
import folium.plugins as plug
marker_cluster = plug.MarkerCluster().add_to(map_osm) # 가까운 거리의 Marker들을 군집시켜서 해당 건수를 표현

for i in range(len(x)):

    folium.Marker([x[i],y[i]], popup='용산CCTV_%d'%i, icon=folium.Icon(color='red', icon='facetime-video')).add_to(marker_cluster)

for i in range(len(x1)):
    
    folium.Marker([x1[i],y1[i]], popup= name[i], icon=folium.Icon(color='orange', icon='star')).add_to(marker_cluster)

[아이콘 종류](https://getbootstrap.com/docs/3.3/components/#glyphicons)

In [49]:
# map저장
map_osm.save('pydata/YONGSAN.html')